In [18]:
CWD = %pwd
CWD = str(CWD)
import sys
import json
from six.moves import cPickle
import os
import statistics
from collections import defaultdict,Iterable

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import numpy as np
from scipy import misc, polyfit, stats
import matplotlib.pyplot as plt
import PIL
# from PIL import Image
%matplotlib inline  
import re

COCO_PATH = os.path.join(CWD,'data','coco')
COCO_ANNO_PATH = os.path.join(COCO_PATH, 'annotations')
sys.path.insert(0, os.path.join(CWD, 'coco', 'PythonAPI'))
from pycocotools.coco import COCO,mask
coco = COCO(os.path.join(COCO_ANNO_PATH,'instances_train2014.json'))

loading annotations into memory...
Done (t=26.83s)
creating index...
index created!


In [2]:
"""
Load Coco data
"""

# load the data from captions
with open(COCO_ANNO_PATH + '/captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = str(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = str(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)
    img_captions[mid]['captions'].append(cap_info['caption'])

In [3]:
from nltk.corpus import stopwords
sw = [x.upper() for x in stopwords.words('english')]

caps = [ c.replace("."," ").replace(","," ").upper() for _,x in img_captions.iteritems() for c in x['captions']]
break_words = " ".join(caps).split(" ")
all_words = list(set([x for x in break_words if x.isalnum() and x not in sw]))
assert(len(all_words) < 22775)
print len(all_words)



22653


In [6]:
word_idx = {}
for idx in range(len(all_words)):
    word_idx[all_words[idx]] = idx
assert(len(word_idx) == len(all_words))

In [5]:
"""
1. Count word_idx -> frequency
2. Count word_idx_1, word_idx_2 -> coexistence frequency
3. Compute conditional probablity: (coexistence frenquency)/(word frequency)
"""
freq   = np.zeros(len(all_words),                   dtype=np.float64)
comatx = np.zeros((len(all_words), len(all_words)), dtype=np.float64)
for _, data in img_captions.iteritems():
    for cap in data['captions']:
        # get all the word ides
        cap = cap.replace("."," ").replace(","," ").upper()
        cap_words_idx = set([word_idx[x] for x in cap.split(" ") if x.isalnum() and x not in sw])
        
        # increment frequency map
        for widx in cap_words_idx:
            freq[widx] += 1
            
        # increment coexistence map
        for widx in cap_words_idx:
            for vidx in cap_words_idx:
                comatx[widx,vidx] += 1
            
prob_mat = comatx / freq

assert(np.allclose(np.diagonal(prob_mat), np.ones(len(all_words))))
assert(len(freq[freq>0]) == len(all_words))

IOError: [Errno 28] No space left on device

In [7]:
def cond_prob(w1, given = None):
    if given == None: return 1
    w1idx = word_idx[w1.upper()]
    w2idx = word_idx[given.upper()]
    return prob_mat[w1idx, w2idx]

# high_cond_prob = [x for x in all_words for y in all_words if x != y and cond_prob(x,given=y) > 0.8]
# len(high_cond_prob)
# print np.mean(prob_mat), np.sum(freq), np.sum(comatx), np.mean(prob_mat)
# print len(prob_mat[prob_mat>0.0])
(len(comatx[comatx > 0.999999]) - len(all_words)) /2

1034533

In [8]:
print cond_prob("winnie", given = "pooh")
print cond_prob("red", given = "rose"), cond_prob("rose", given = "red")
# all_words[10]
print freq[word_idx['winnie'.upper()]], freq[word_idx['pooh'.upper()]]
print freq[word_idx['red'.upper()]],    freq[word_idx['rose'.upper()]]
print comatx[word_idx['RED'], word_idx['ROSE']], comatx[word_idx['ROSE'], word_idx['RED']]


0.785714285714
0.177631578947 0.00227560050569
11.0 14.0
11865.0 152.0
27.0 27.0


In [9]:
N = len(all_words)
prob_mat_not_sym = (np.ones((N,N)) - np.diag(np.ones(N))) * prob_mat
ones_idx = np.where(prob_mat_not_sym > 0.9)
print len(ones_idx[0])

49824


In [10]:
def wanted(x,y):
    return not (freq[y] / freq[x] > 100 or freq[x] / freq[y] > 100 or freq[y] < 10 or freq[x] < 10)
clean_idx = [(x,y) for x,y in zip(ones_idx[0], ones_idx[1]) if wanted(x,y)]
print len(clean_idx)

84


In [11]:
import random
# idx = random.randint(0, len(clean_idx[0]));
for idx in range(len(clean_idx)):
    x, y = clean_idx[idx]
    print x, y
    print all_words[x], all_words[y], freq[x], freq[y]
    print cond_prob(all_words[x], given=all_words[y]), cond_prob(all_words[y], given=all_words[x])


446 21910
MOUSE MICKEY 1130.0 21.0
0.952380952381 0.0176991150442
470 11990
LIT DIMLY 987.0 154.0
0.961038961039 0.149949341439
1309 10
POOH WINNIE 14.0 11.0
1.0 0.785714285714
1326 6462
TENNIS RACKETS 11224.0 440.0
0.920454545455 0.0360833927299
1326 7618
TENNIS RACKET 11224.0 3102.0
0.930689877498 0.257216678546
1326 19165
TENNIS COURT 11224.0 4320.0
0.938194444444 0.361101211689
1326 21593
TENNIS RACQUET 11224.0 1553.0
0.938184159691 0.129811119031
1765 10336
TATER TOTS 17.0 15.0
1.0 0.882352941176
2035 8835
ROOM LIVING 12730.0 5420.0
0.935793357934 0.398428908091
2059 14504
WIND INDICATOR 237.0 12.0
0.916666666667 0.0464135021097
2231 19930
WEATHER VEIN 144.0 10.0
1.0 0.0694444444444
2231 22345
WEATHER VANE 144.0 37.0
0.945945945946 0.243055555556
3072 15422
PING PONG 28.0 29.0
0.965517241379 1.0
3188 10894
IRON WROUGHT 167.0 26.0
1.0 0.155688622754
4082 17614
IMAGE UNABLE 2781.0 58.0
0.948275862069 0.019777058612
4277 15053
WHEEL FERRIS 242.0 17.0
1.0 0.0702479338843
4988 1020
KRI

In [12]:
def most_prob_words(word, num = 10):
    idxes = np.argsort(prob_mat[:,word_idx[word.upper()]])
    words = [all_words[idx] for idx in idxes[:-num:-1]]
    return words

most_prob_words("standing")

[u'STANDING',
 u'MAN',
 u'NEXT',
 u'PEOPLE',
 u'TWO',
 u'FIELD',
 u'WOMAN',
 u'GROUP',
 u'HOLDING']

In [13]:
all_info = json.load(open(os.path.join(COCO_ANNO_PATH, 'image_info_test2014.json')))
category_names = [x['name'] for x in all_info['categories']]
categories = {x['name']:{} for x in all_info['categories']}
for cate, d in categories.iteritems():
    for word in cate.split(" "):
        d.update({ word : most_prob_words(word)})
print category_names

[u'person', u'bicycle', u'car', u'motorcycle', u'airplane', u'bus', u'train', u'truck', u'boat', u'traffic light', u'fire hydrant', u'stop sign', u'parking meter', u'bench', u'bird', u'cat', u'dog', u'horse', u'sheep', u'cow', u'elephant', u'bear', u'zebra', u'giraffe', u'backpack', u'umbrella', u'handbag', u'tie', u'suitcase', u'frisbee', u'skis', u'snowboard', u'sports ball', u'kite', u'baseball bat', u'baseball glove', u'skateboard', u'surfboard', u'tennis racket', u'bottle', u'wine glass', u'cup', u'fork', u'knife', u'spoon', u'bowl', u'banana', u'apple', u'sandwich', u'orange', u'broccoli', u'carrot', u'hot dog', u'pizza', u'donut', u'cake', u'chair', u'couch', u'potted plant', u'bed', u'dining table', u'toilet', u'tv', u'laptop', u'mouse', u'remote', u'keyboard', u'cell phone', u'microwave', u'oven', u'toaster', u'sink', u'refrigerator', u'book', u'clock', u'vase', u'scissors', u'teddy bear', u'hair drier', u'toothbrush']


In [14]:
categories[category_names[random.randint(0, len(category_names))]]

{u'bicycle': [u'BICYCLE',
  u'MAN',
  u'RIDING',
  u'STREET',
  u'PARKED',
  u'NEXT',
  u'WOMAN',
  u'PERSON',
  u'FRONT']}

In [27]:
import scipy
with open(os.path.join(CWD,'stats','cond_prob','prob_mat.npz'),'w+') as f:
    np.save(f, np.csc_matrix(prob_mat))

AttributeError: 'module' object has no attribute 'csc_matrix'

In [23]:
with open(os.path.join(CWD,'stats','cond_prob','freq.npz'),'w+') as f:
    np.save(f, freq)

In [24]:
with open(os.path.join(CWD,'stats','cond_prob','comatx.npz'),'w+') as f:
    np.save(f, np.csc_matrix(comatx))

In [26]:
with open(os.path.join(CWD,'stats','cond_prob','words_and_rvwords.pkl'),'w+') as f:
    cPickle.dump(all_words, f, protocol=cPickle.HIGHEST_PROTOCOL)
